In [19]:
using FileIO, Images
const tiffFile = "/gpfs/data/shohamlab/shared_data/jon_2p_data/JG1150/190711/aligned/JG1150_190711_field1_beh_00001_00001.tif"
tiff = load(tiffFile)

512×512×2000 Array{Gray{N0f16},3} with eltype Gray{Normed{UInt16,16}}:
[:, :, 1] =
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)      …  Gray{N0f16}(9.0e-5) 
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0)    
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0007) 
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.00421)
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0)    
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)      …  Gray{N0f16}(0.00075)
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.00043)
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0)    
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.00063)
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.00246)
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)      …  Gray{N0f16}(0.0)    
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0)    
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0)    
 ⋮                                       ⋱                      
 Gray{N

In [76]:
using Glob
movFiles = sort(glob("JG1150_190711_field1_beh_00001_*.tif","/gpfs/data/shohamlab/shared_data/jon_2p_data/JG1150/190711/aligned/"))
mov = cat([Gray.(load(x)) for x in movFiles]...,dims=3)

512×512×14112 Array{Gray{N0f16},3} with eltype Gray{Normed{UInt16,16}}:
[:, :, 1] =
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)      …  Gray{N0f16}(9.0e-5) 
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0)    
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0007) 
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.00421)
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0)    
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)      …  Gray{N0f16}(0.00075)
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.00043)
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0)    
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.00063)
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.00246)
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)      …  Gray{N0f16}(0.0)    
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0)    
 Gray{N0f16}(0.0)  Gray{N0f16}(0.0)         Gray{N0f16}(0.0)    
 ⋮                                       ⋱                      
 Gray{

In [48]:
const maskDir = "/gpfs/data/shohamlab/shared_data/jon_2p_data/JG1150/190711/masks_improved_002"
maskFiles = joinpath.(abspath(maskDir), sort(readdir(maskDir)))
nMasks = length(maskFiles)
exMask = Gray.(load(maskFiles[1]))
height,width = size(exMask)
masks = Array{Gray{Normed{UInt8,8}},3}(undef,(nMasks, height,width));
for i=1:length(maskFiles)
   masks[i,:,:] = Gray.(load(maskFiles[i]))
end

In [67]:
using TensorOperations
function maskMult(mov::Array{Gray{Normed{UInt16,16}},3}, masks::Array{Gray{Normed{UInt8,8}},3} )
    nMasks = size(masks,1)
    T = size(mov,3)
    F = Array{Float32,2}(undef,(nMasks,T))
    @tensor begin
       F[nm,t] = masks[nm,h,w]*mov[h,w,t]
    end
    return F
end

maskMult (generic function with 1 method)

In [ ]:
@time F = maskMult(mov,masks)

In [77]:
size(tiff)

(512, 512, 2000)